In [1]:
# Standard data manipulation libraries
import pandas as pd
import numpy as np

# Webscraping imports
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [2]:
# Test with random sample of 1000 ssvids
sample_ssvid = pd.read_csv('./data/sample_ssvid.csv')

In [3]:
# convert ssvid column to list of numbers
ssvids = sample_ssvid['ssvid'].values.tolist()
ssvids[:10]

[247046810,
 482510000,
 412467914,
 440129410,
 412322973,
 416000679,
 316033397,
 251258540,
 412443871,
 228330000]

In [4]:
test = [23510249, 900]

Is able to loop through multiple search results in the table but when given a MMSI with one number cannot return to enter the next number.

In [5]:
# define chromedriver location
chromedriver = '/Users/jessicaertel/Downloads/chromedriver'

# define driver object
driver = webdriver.Chrome(chromedriver)

# Open browser
driver.get('https://webaccounts.imo.org')

# Select 'Public' in drop down menu
authority = Select(driver.find_element_by_name('ctl00$cpMain$ddlAuthorityType'))
authority.select_by_value('PUBLIC')

# Enter username and password
driver.find_element_by_id("ctl00_cpMain_txtUsername").send_keys("jessicarose")
driver.find_element_by_id("ctl00_cpMain_txtPassword").send_keys("Fishing0")

# Click log in button
driver.find_element_by_id("ctl00_cpMain_btnLogin").click()

# Navigate to Global Integrated Shipping Information System
driver.find_element_by_id("ctl00_cpMain_lnkGISIS").click()
driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr/td/div[3]/a/div[2]/span[1]').click()

# Navigate to Ship and Company Particulars - if intermediate page does not appear
#driver.find_by_link_text('/Public/SHIPS/Default.aspx').click()

# Create lists to hold scraped data
name, imo_num, flag, call_sign, mmsi, vessel_type, built, tonnage, owner = [],[],[],[],[],[],[],[],[]

# For each number in ssvids
for num in test:
    
    # Enter number into MMSI field and click search
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").send_keys(num)
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_btnSearchShips").click()
    driver.implicitly_wait(5)
    
    # define the results table
    table = driver.find_element_by_id('ctl00_bodyPlaceHolder_Default_gridShips')
    rows = table.find_elements(By.TAG_NAME, "tr")
    
    if len(rows) == 1:
        WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH,f'/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr[{row}]')))
        driver.find_element_by_xpath('//*[@id="ctl00_bodyPlaceHolder_Default_gridShips"]/tbody/td').click()
        
        # scrape and append the data into lists
        name.append(driver.find_element_by_id("sNameCur").text)
        imo_num.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[2]/td[2]").text)
        flag.append(driver.find_element_by_id("sFlagCur").text)
        call_sign.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[4]/td[2]").text)
        mmsi.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[5]/td[2]").text)
        vessel_type.append(driver.find_element_by_id("sTypeCur").text)
        built.append(driver.find_element_by_id("sBuildDateCur").text)
        tonnage.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[2]/tbody/tr[3]/td[2]").text)
        owner.append(driver.find_element_by_id("sRegOwnerCur").text)
        time.sleep(1)
        
        # return to the search results page to scrape next result
        driver.execute_script("window.history.go(-1)")
        driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()
        
    for row in range(1, len(rows)+1):
        # wait until the full table appears
        WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH,f'/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr[{row}]')))
        driver.find_element_by_xpath(f'//*[@id="ctl00_bodyPlaceHolder_Default_gridShips"]/tbody/tr[{row}]').click()
        
        # scrape and append the data into lists
        name.append(driver.find_element_by_id("sNameCur").text)
        imo_num.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[2]/td[2]").text)
        flag.append(driver.find_element_by_id("sFlagCur").text)
        call_sign.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[4]/td[2]").text)
        mmsi.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[5]/td[2]").text)
        vessel_type.append(driver.find_element_by_id("sTypeCur").text)
        built.append(driver.find_element_by_id("sBuildDateCur").text)
        tonnage.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[2]/tbody/tr[3]/td[2]").text)
        owner.append(driver.find_element_by_id("sRegOwnerCur").text)
        time.sleep(1)
        
        # return to the search results page to scrape next result
        driver.execute_script("window.history.go(-1)")
        
    # Clear search field
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()

KeyboardInterrupt: 

Stale Element Exception - seems like the DOM expires when we go back to the search results and loop cannot find the next element?

```python

# define chromedriver location
chromedriver = '/Users/jessicaertel/Downloads/chromedriver'

# define driver object
driver = webdriver.Chrome(chromedriver)

# Open browser
driver.get('https://webaccounts.imo.org')

# Select 'Public' in drop down menu
authority = Select(driver.find_element_by_name('ctl00$cpMain$ddlAuthorityType'))
authority.select_by_value('PUBLIC')

# Enter username and password
driver.find_element_by_id("ctl00_cpMain_txtUsername").send_keys("jessicarose")
driver.find_element_by_id("ctl00_cpMain_txtPassword").send_keys("Fishing0")

# Click log in button
driver.find_element_by_id("ctl00_cpMain_btnLogin").click()

# Navigate to Global Integrated Shipping Information System
driver.find_element_by_id("ctl00_cpMain_lnkGISIS").click()
driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr/td/div[3]/a/div[2]/span[1]').click()

# Navigate to Ship and Company Particulars - if intermediate page does not appear
#driver.find_by_link_text('/Public/SHIPS/Default.aspx').click()

# Create lists to hold scraped data
name, imo_num, flag, call_sign, mmsi, vessel_type, built, tonnage, owner = [],[],[],[],[],[],[],[],[]
        
# For each number in ssvids
for num in test:
    
    # Enter number into MMSI field and click search
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").send_keys(num)
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_btnSearchShips").click()
    driver.implicitly_wait(5)
    
    # define the results table
    table = driver.find_element_by_id('ctl00_bodyPlaceHolder_Default_gridShips')
    rows = table.find_elements(By.TAG_NAME, "tr") 
    #rows = driver.find_elements_by_class_name("gridviewer_row")
    #rows = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "tr")))
    
    for row in range(len(rows)):
        i = 1
        # wait until the full table appears
        WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH,f'/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr[{i}]')))
        driver.find_element_by_xpath(f'//*[@id="ctl00_bodyPlaceHolder_Default_gridShips"]/tbody/tr[{i}]').click()
        i += 1    
    
    # click on each row in the table
    for row in rows:
        print(len(rows))

        #wait until the full table appears
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,'/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr[2]')))
        row.click()

        # scrape and append the data into lists
        name.append(driver.find_element_by_id("sNameCur").text)
        imo_num.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[2]/td[2]").text)
        flag.append(driver.find_element_by_id("sFlagCur").text)
        call_sign.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[4]/td[2]").text)
        mmsi.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[5]/td[2]").text)
        vessel_type.append(driver.find_element_by_id("sTypeCur").text)
        built.append(driver.find_element_by_id("sBuildDateCur").text)
        tonnage.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[2]/tbody/tr[3]/td[2]").text)
        owner.append(driver.find_element_by_id("sRegOwnerCur").text)

        # return to the search results page to scrape next result
        driver.execute_script("window.history.go(-1)")
        
    # Clear search field
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()
```

In [ ]:
print(name)

In [ ]:
print(imo_num)

This code works for scraping the single search results, but isn't able to loop through the table.

## Old Code - DO NOT RUN
```python
# define chromedriver location
chromedriver = '/Users/jessicaertel/Downloads/chromedriver'

# define driver object
driver = webdriver.Chrome(chromedriver)

# Open browser
driver.get('https://webaccounts.imo.org')

# Select 'Public' in drop down menu
authority = Select(driver.find_element_by_name('ctl00$cpMain$ddlAuthorityType'))
authority.select_by_value('PUBLIC')

# Enter username and password
driver.find_element_by_id("ctl00_cpMain_txtUsername").send_keys("jessicarose")
driver.find_element_by_id("ctl00_cpMain_txtPassword").send_keys("Fishing0")

# Click log in button
driver.find_element_by_id("ctl00_cpMain_btnLogin").click()

# Navigate to Global Integrated Shipping Information System
driver.find_element_by_id("ctl00_cpMain_lnkGISIS").click()
driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr/td/div[3]/a/div[2]/span[1]').click()

# Navigate to Ship and Company Particulars - if intermediate page does not appear
#driver.find_by_link_text('/Public/SHIPS/Default.aspx').click()

# Create lists to hold scraped data
name, imo_num, flag, call_sign, mmsi, vessel_type, built, tonnage, owner = [],[],[],[],[],[],[],[],[]
        
# For each number in ssvids
for num in test:
    
    # Enter number into MMSI field and click search
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").send_keys(num)
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_btnSearchShips").click()
    driver.implicitly_wait(5)
    
    # if one result is displayed
    driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr').click()
    
    # scrape and append the data into lists
    name.append(driver.find_element_by_id("sNameCur").text)
    imo_num.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[2]/td[2]").text)
    flag.append(driver.find_element_by_id("sFlagCur").text)
    call_sign.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[4]/td[2]").text)
    mmsi.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[5]/td[2]").text)
    vessel_type.append(driver.find_element_by_id("sTypeCur").text)
    built.append(driver.find_element_by_id("sBuildDateCur").text)
    tonnage.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[2]/tbody/tr[3]/td[2]").text)
    owner.append(driver.find_element_by_id("sRegOwnerCur").text)
        
    # return to the search results page to scrape next result
    driver.execute_script("window.history.go(-1)")
    
    # if multiple search results are displayed
    results = driver.find_elements_by_class_name('gridviewer_row tipsterise noline tooltipstered')
    if len(results) >= 1:
        for i in range(len(results)):
            driver.find_element_by_class_name('gridviewer_row tipsterise noline tooltipstered').click()

            # scrape and append the data into lists
            name.append(driver.find_element_by_id("sNameCur").text)
            imo_num.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[2]/td[2]").text)
            flag.append(driver.find_element_by_id("sFlagCur").text)
            call_sign.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[4]/td[2]").text)
            mmsi.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[5]/td[2]").text)
            vessel_type.append(driver.find_element_by_id("sTypeCur").text)
            built.append(driver.find_element_by_id("sBuildDateCur").text)
            tonnage.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[2]/tbody/tr[3]/td[2]").text)
            owner.append(driver.find_element_by_id("sRegOwnerCur").text)
        
            # return to the search results page to scrape next result
            driver.execute_script("window.history.go(-1)")
        
    # Clear search field
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()
```

In [ ]:
print(name)

In [ ]:
print(imo_num)

This option tries to classify the length of results, and if the length is larger than 0, loop through each row and pull the data. Issue: classifying results as non-results or not clicking the result if it does appear.

In [ ]:
# define chromedriver location
chromedriver = '/Users/jessicaertel/Downloads/chromedriver'

# define driver object
driver = webdriver.Chrome(chromedriver)

# Open browser
driver.get('https://webaccounts.imo.org')

# Select 'Public' in drop down menu
authority = Select(driver.find_element_by_name('ctl00$cpMain$ddlAuthorityType'))
authority.select_by_value('PUBLIC')

# Enter username and password
driver.find_element_by_id("ctl00_cpMain_txtUsername").send_keys("jessicarose")
driver.find_element_by_id("ctl00_cpMain_txtPassword").send_keys("Fishing0")

# Click log in button
driver.find_element_by_id("ctl00_cpMain_btnLogin").click()

# Navigate to Global Integrated Shipping Information System
driver.find_element_by_id("ctl00_cpMain_lnkGISIS").click()
driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr/td/div[3]/a/div[2]/span[1]').click()

# Navigate to Ship and Company Particulars - if intermediate page does not appear
#driver.find_by_link_text('/Public/SHIPS/Default.aspx').click()

# Create lists to hold scraped data
name, imo_num, flag, call_sign, mmsi, vessel_type, built, tonnage, owner = [],[],[],[],[],[],[],[],[]
        
# For each number in ssvids
for num in ssvids:
    
    # Enter number into MMSI field and click search
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").send_keys(num)
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_btnSearchShips").click()
    driver.implicitly_wait(5)
    
    # Define the results table
    #result = driver.find_elements_by_id("ctl00_bodyPlaceHolder_Default_gridShips")
    #result = driver.find_elements_by_class_name('gridviewer_row tipsterise noline tooltipstered')
    
    # Define how many results in the table (this only picks up green rows)
    rows = driver.find_elements_by_class_name('gridviewer_row tipsterise noline tooltipstered')
    
    # if no search result is displayed, remove text and input next number
    if len(rows) == 0:
        print(rows)
        #driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()
    
    # if one/multiple search results are displayed
    elif len(rows) >= 1:
        
        # loop through each element in the result page
        for i in range(len(rows)):
            driver.find_element_by_class_name('gridviewer_row tipsterise noline tooltipstered').click()

            # scrape and append the data into lists
            name.append(driver.find_element_by_id("sNameCur"))
            imo_num.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[2]/td[2]"))
            flag.append(driver.find_element_by_id("sFlagCur"))
            call_sign.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[4]/td[2]"))
            mmsi.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[5]/td[2]"))
            vessel_type.append(driver.find_element_by_id("sTypeCur"))
            built.append(driver.find_element_by_id("sBuildDateCur"))
            tonnage.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[2]/tbody/tr[3]/td[2]"))
            owner.append(driver.find_element_by_id("sRegOwnerCur"))
        
            # return to the search results page to scrape next result
            driver.execute_script("window.history.go(-1)")
        
    # Clear search field
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()

This option will classify two outcomes into results vs. not results and attempt to click on the first result if it appears.

In [ ]:
# Define two outcomes from search
    no_result = EC.presence_of_element_located((By.XPATH, "/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr/td/p/a"))
    result = EC.presence_of_element_located((By.XPATH, "/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr[1]"))
   
    result = driver.find_elements_by_class_name('gridviewer_row tipsterise noline tooltipstered')
    no_result = driver.find_element_by_class_name('gridviewer_emptydatarow') #seems to break once result appears
    
#     no_result = driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr')
#     result = driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/thead/tr')
#     result = driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_gridShips")
    
    print(no_result)
    print(result)
    
    # if no search result is displayed, remove text and add the next number
    if no_result:
        driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()

    # if there is a search result click on it
    # change to elements and look at the length of the list (will return the html element)
    elif result: 
        driver.find_element_by_class_name('gridviewer_row tipsterise noline tooltipstered').click()
        
        # scrape and append data to each list
        name.append(driver.find_element_by_id("sNameCur"))
        imo_num.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[2]/td[2]"))
        flag.append(driver.find_element_by_id("sFlagCur"))
        call_sign.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[4]/td[2]"))
        mmsi.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[5]/td[2]"))
        vessel_type.append(driver.find_element_by_id("sTypeCur"))
        built.append(driver.find_element_by_id("sBuildDateCur"))
        tonnage.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[2]/tbody/tr[3]/td[2]"))
        owner.append(driver.find_element_by_id("sRegOwnerCur"))
        
        # return to the search page and clear search
        driver.execute_script("window.history.go(-1)")
        driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()

In [ ]:
# transform lists into pandas dataframe
imo = pd.DataFrame({'name': name,
                     'imo_num': imo_num,
                     'flag': flag,
                     'call_sign': call_sign,
                     'mmsi': mmsi,
                     'vessel_type': vessel_type,
                     'built' : built,
                     'tonnage': tonnage,
                     'owner': owner})

## References
- https://stackoverflow.com/questions/60362018/macos-catalinav-10-15-3-error-chromedriver-cannot-be-opened-because-the-de
- https://selenium-python.readthedocs.io/locating-elements.html
- https://stackoverflow.com/questions/27338742/how-do-i-send-a-delete-keystroke-to-a-text-field-using-selenium-with-python